# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [1]:
$:.unshift "."
require 'db_setup'
require 'db_load'


==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigration: migrated (0.0000s) ===================================

.................................................................................................

true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [2]:
book = Book.first
puts book.title
puts book.author


Végjáték (regény)
#<Author:0x00000003f1f8d0>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [3]:


book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"


Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [4]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end


Anna Karenina
Azazel (powieść)
Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 30, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [5]:
IRuby.html "<h1>Test HTML-a</h1>"


"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [6]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"


"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [7]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document


"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [75]:
authors = Author.where('born < ?', Date.parse("1900-01-01")).where("born is not null")
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end


Juliusz Słowacki 04-09-1809
Henryk Sienkiewicz 05-05-1846
Eliza Orzeszkowa 06-06-1841
Adam Mickiewicz 24-12-1789
Jack London 12-01-1876
Mikołaj Kopernik 19-02-1473
Maria Franciszka Kozłowska 27-05-1862
William Szekspir 23-04-1564
John Ronald Reuel Tolkien 03-01-1892
Aleksander Dumas (ojciec) 24-07-1802
Tadeusz Dołęga-Mostowicz 10-08-1898
Vladimir Nabokov 22-04-1899
Adolf Hitler 20-04-1889
Benedykt Chmielowski 20-03-1700
Aldous Huxley 26-07-1894
Honoré de Balzac 20-05-1799
Franz Kafka 03-07-1883
Howard Phillips Lovecraft 20-08-1890
Lew Tołstoj 09-09-1828
Ilja Ilf 15-10-1897
Gabriela Zapolska 30-03-1857
Clive Staples Lewis 29-11-1898
Iwan Turgieniew 09-11-1818
Lucy Maud Montgomery 30-11-1874
Aleksander Majkowski 17-07-1876


#<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,

którzy zmarli w XX wieku.

In [85]:
def display_authors(author)
  "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")}"
end
Author.where(died: (Date.parse("1901-1-1")..Date.parse("2001-1-1"))).each do |author|
  puts display_authors(author)
end



Henryk Sienkiewicz 05-05-1846 05-11-1916
Eliza Orzeszkowa 06-06-1841 23-05-1910
Jack London 12-01-1876 22-11-1916
Antoni Gołubiew 25-02-1907 27-06-1979
George Orwell 25-06-1903 21-01-1950
John Ronald Reuel Tolkien 03-01-1892 02-09-1973
Tadeusz Dołęga-Mostowicz 10-08-1898 20-09-1939
Vladimir Nabokov 22-04-1899 02-07-1977
Adolf Hitler 20-04-1889 30-04-1945
Aldous Huxley 26-07-1894 22-11-1963
Franz Kafka 03-07-1883 03-06-1924
Howard Phillips Lovecraft 20-08-1890 15-03-1937
Karol Olgierd Borchardt 25-03-1905 20-05-1986
Lew Tołstoj 09-09-1828 20-11-1910
Janusz Zajdel 15-08-1938 19-07-1985
Isaac Asimov 04-10-1919 06-04-1992
Ilja Ilf 15-10-1897 13-04-1937
Jewgienij Pietrow (pisarz) 13-12-1903 02-07-1942
Gabriela Zapolska 30-03-1857 17-12-1921
Clive Staples Lewis 29-11-1898 22-11-1963
Lucy Maud Montgomery 30-11-1874 24-04-1942
Alfred Szklarski 21-01-1912 09-04-1992
Aleksander Majkowski 17-07-1876 10-02-1938
Ian Fleming 28-05-1908 12-08-1964


[#<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 8, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 11, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 14, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 17, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 19, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http:

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [20]:
Author.where("image_url is not null").order(:born).last(5).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end

Orson Scott Card 1951-08-24
Boris Akunin 1956-05-20
Rafał A. Ziemkiewicz 1964-09-13
J. K. Rowling 1965-07-31
Andrzej Pilipiuk 1974-03-20


[#<Author id: 6, name: "Orson Scott", surname: "Card", born: "1951-08-24", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 31, name: "Boris", surname: "Akunin", born: "1956-05-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 50, name: "Rafał A.", surname: "Ziemkiewicz", born: "1964-09-13", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 16, name: "J. K.", surname: "Rowling", born: "1965-07-31", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 41, name: "Andrzej", surname: "Pilipiuk", born: "1974-03-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [86]:
def display_book(book)
  if book.published != nil
  "<h1>#{book.title} #{book.published}</h1><img src='#{book.image_url}'/>"
  else
    "<h1>#{book.title} Brak danych publikacji</h1><img src='#{book.image_url}'/>" 
  end
end

book = Book.order(:published).where("published is not null and published != 0").first
document = display_book(book)

IRuby.html document


"<h1>De revolutionibus orbium coelestium 1543</h1><img src=''/>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [29]:
book = Book.where("published is null or published == 0")
book.each do |book|
  puts "#{book.title} #{book.author.name} #{book.author.surname} #{book.published}"
end


Végjáték (regény) Orson Scott Card 
Hamlet William Szekspir 
Ojcowie i dzieci Iwan Turgieniew 0


[#<Book id: 1, title: "Végjáték (regény)", language: nil, author_id: 6, published: nil, image_url: nil>, #<Book id: 10, title: "Hamlet", language: nil, author_id: 15, published: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Book id: 65, title: "Ojcowie i dzieci", language: nil, author_id: 43, published: 0, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [38]:
def display_author(author)
  author = Author.where(surname: author).first
  books = author.books
  html= "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li>#{book.title} #{book.published}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document


"<h1>Stanisław Lem</h1><ul><li>Bajki robotów 1964</li><li>Solaris (powieść) 1961</li><li>Bezsenność (zbiór opowiadań) 1971</li><li>Głos Pana 1968</li><li>Inwazja z Aldebarana (zbiór opowiadań) 1959</li><li>Katar (powieść) 1976</li><li>Księga robotów 1961</li><li>Maska (zbiór dzieł) 1976</li><li>Niezwyciężony i inne opowiadania 1964</li><li>Noc księżycowa 1963</li><li>Pamiętnik znaleziony w wannie 1961</li><li>Polowanie (zbiór opowiadań) 1965</li><li>Powtórka (zbiór dzieł) 1979</li><li>Pożytek ze smoka 1993</li><li>Rasa drapieżców. Teksty ostatnie 2006</li><li>Sezam i inne opowiadania 1954</li><li>Summa technologiae 1964</li><li>Wizja lokalna (powieść) 1982</li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [39]:
def display_author(author)
  author = Author.where(surname: author).first
  books = author.books.where("title like '%zbiór%'")
  html= "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li>#{book.title} #{book.published}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document


"<h1>Stanisław Lem</h1><ul><li>Bezsenność (zbiór opowiadań) 1971</li><li>Inwazja z Aldebarana (zbiór opowiadań) 1959</li><li>Maska (zbiór dzieł) 1976</li><li>Polowanie (zbiór opowiadań) 1965</li><li>Powtórka (zbiór dzieł) 1979</li></ul>"

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [44]:
def display_author(author)
  author = Author.where(surname: author).first
  books = author.books.where("published = ?",author.died.year)
  html= "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li>#{book.title} #{book.published}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li>Rasa drapieżców. Teksty ostatnie 2006</li></ul>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [53]:
def display_books()
  books = Book.all
  ksiazka = "<h1>Lista książek</h1>"
  books.each do |book|
    ksiazka << "<li>"
    ksiazka << "#{book.title} #{book.author.name} #{book.author.surname} #{book.published}"
    if (book.image_url != nil)
      ksiazka << " <img src='#{book.image_url}'/>"
    end
  end
  ksiazka << "</li>"
end

document = display_books()
IRuby.html document


"<h1>Lista książek</h1><li>Végjáték (regény) Orson Scott Card <li>Pani Jeziora (powieść) Andrzej Sapkowski 1999<li>Bellew Zawierucha Jack London 1912<li>Fałszywy trop (powieść) Henning Mankell 1995<li>Bajki robotów Stanisław Lem 1964<li>Bolesław Chrobry (powieść) Antoni Gołubiew 1947<li>De revolutionibus orbium coelestium Mikołaj Kopernik 1543<li>Dzieło Wielkiego Miłosierdzia Maria Franciszka Kozłowska 1922<li>Folwark zwierzęcy George Orwell 1945 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Animalism_flag.svg?width=300'/><li>Hamlet William Szekspir  <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/><li>Harry Potter J. K. Rowling 1997 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Harry_Potter_wordmark.svg?width=300'/><li>Hobbit, czyli tam i z powrotem John Ronald Reuel Tolkien 1937<li>Hrabia Monte Christo Aleksander Dumas (ojciec) 1844 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Louis_Français-Dantès_sur_son_rocher.jpg?width=300'/><li>Kariera Nikodema Dyzmy Tadeusz Dołęga-Mostowicz 1932<li>Krzyżacy (powieść) Henryk Sienkiewicz 1900 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/PL_Henryk_Sienkiewicz-Krzyżacy_0005.jpeg?width=300'/><li>Lolita Vladimir Nabokov 1955<li>Mein Kampf Adolf Hitler 1925 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Landsberg.Jail.jpg?width=300'/><li>Nad Niemnem Eliza Orzeszkowa 1888 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nad_Niemnem_-_manuscript.jpg?width=300'/><li>Niedokończone opowieści Christopher Tolkien 1980<li>Nowe Ateny Benedykt Chmielowski 1745 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nowe_ateny1.jpg?width=300'/><li>Nowy wspaniały świat Aldous Huxley 1932<li>Ogniem i mieczem Henryk Sienkiewicz 1884 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Podbipieta.JPG?width=300'/><li>Ojciec Goriot Honoré de Balzac 1835 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Father_Goriot_by_H._Daumier_(1842).JPG?width=300'/><li>Proces (powieść) Franz Kafka 1925 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka_Der_Prozess_1925.jpg?width=300'/><li>Psałterz Dawidów Jan Kochanowski 1579 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kochanowski_-_Psałterz_Dawidów_(1579).jpg?width=300'/><li>Quo vadis Henryk Sienkiewicz 1896 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rzym_popiersie_Sienkiewicza_w_kosciele_Domine_Quo_Vadis.JPG?width=300'/><li>Romeo i Julia William Szekspir 1597 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Romeo_and_juliet_brown.jpg?width=300'/><li>Silmarillion John Ronald Reuel Tolkien 1977 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Silmarrillion,_Just_under_the_Cover.jpg?width=300'/><li>Solaris (powieść) Stanisław Lem 1961 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Solaris,_various_editions_02.jpg?width=300'/><li>Władca Pierścieni John Ronald Reuel Tolkien 1954 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Unico_Anello.png?width=300'/><li>Zew Cthulhu (opowiadanie) Howard Phillips Lovecraft 1928<li>Znaczy Kapitan Karol Olgierd Borchardt 1960<li>Anna Karenina Lew Tołstoj 1877 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/AnnaKareninaTitle.jpg?width=300'/><li>Azazel (powieść) Boris Akunin 1998<li>Bezsenność (zbiór opowiadań) Stanisław Lem 1971<li>Cała prawda o planecie Ksi Janusz Zajdel 1983<li>Dekorator Boris Akunin 1998<li>Diamentowa karoca Boris Akunin 2003<li>Dives in misericordia Jan Paweł II 1980 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_paul_2_coa.svg?width=300'/><li>Droga na molo w Wigan George Orwell 1937<li>Druga Fundacja Isaac Asimov 1953<li>Dwanaście krzeseł Ilja Ilf 1928 <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Ostap_Bender_Monument_in_Kharkiv.jpg?width=300'/><li>Fundacja (powieść) Isaac Asimov 1951<li>Fundacja i Imperium Isaac Asim

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [78]:
def display_authors()
  html = "<h1>Lista autorów</h1>"
  authors = Author.all
  authors.each do |author|
    html << "<h2>Autor</h2>"
    html << "<li>"
    html << "#{author.name} #{author.surname}"
    if (author.image_url != nil)
      html << " <img src='#{author.image_url}'/>"
    end
    html << "<br />"
    if author.books.empty?
      html << "Brak danych"
    else
     html << "<b>Lista jego/jej książek</b>"
    end
    author.books.each do |book|
        html << "<li> #{book.title} #{book.published} </li>"
    end
  end
  html << "</li>"
end

document = display_authors()
IRuby.html document

"<h1>Lista autorów</h1><h2>Autor</h2><li>Juliusz Słowacki<br />Brak danych<h2>Autor</h2><li>Henryk Sienkiewicz<br /><b>Lista jego/jej książek</b><li> Krzyżacy (powieść) 1900 </li><li> Ogniem i mieczem 1884 </li><li> Quo vadis 1896 </li><h2>Autor</h2><li>Eliza Orzeszkowa<br /><b>Lista jego/jej książek</b><li> Nad Niemnem 1888 </li><h2>Autor</h2><li>Adam Mickiewicz<br /><b>Lista jego/jej książek</b><li> Księgi narodu polskiego i pielgrzymstwa polskiego 1832 </li><h2>Autor</h2><li>Orson Scott Card <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Végjáték (regény)  </li><h2>Autor</h2><li>Andrzej Sapkowski <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Pani Jeziora (powieść) 1999 </li><h2>Autor</h2><li>Jack London <img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/><br /><b>Lista jego/jej książek</b><li> Bellew Zawierucha 1912 </li><h2>Autor</h2><li>Henning Mankell <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Fałszywy trop (powieść) 1995 </li><h2>Autor</h2><li>Stanisław Lem <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Bajki robotów 1964 </li><li> Solaris (powieść) 1961 </li><li> Bezsenność (zbiór opowiadań) 1971 </li><li> Głos Pana 1968 </li><li> Inwazja z Aldebarana (zbiór opowiadań) 1959 </li><li> Katar (powieść) 1976 </li><li> Księga robotów 1961 </li><li> Maska (zbiór dzieł) 1976 </li><li> Niezwyciężony i inne opowiadania 1964 </li><li> Noc księżycowa 1963 </li><li> Pamiętnik znaleziony w wannie 1961 </li><li> Polowanie (zbiór opowiadań) 1965 </li><li> Powtórka (zbiór dzieł) 1979 </li><li> Pożytek ze smoka 1993 </li><li> Rasa drapieżców. Teksty ostatnie 2006 </li><li> Sezam i inne opowiadania 1954 </li><li> Summa technologiae 1964 </li><li> Wizja lokalna (powieść) 1982 </li><h2>Autor</h2><li>Antoni Gołubiew <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Bolesław Chrobry (powieść) 1947 </li><h2>Autor</h2><li>Mikołaj Kopernik <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> De revolutionibus orbium coelestium 1543 </li><h2>Autor</h2><li>Maria Franciszka Kozłowska <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/><br /><b>Lista jego/jej książek</b><li> Dzieło Wielkiego Miłosierdzia 1922 </li><h2>Autor</h2><li>George Orwell <img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Folwark zwierzęcy 1945 </li><li> Droga na molo w Wigan 1937 </li><h2>Autor</h2><li>William Szekspir <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Hamlet  </li><li> Romeo i Julia 1597 </li><li> Sen nocy letniej 1600 </li><h2>Autor</h2><li>J. K. Rowling <img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Harry Potter 1997 </li><h2>Autor</h2><li>John Ronald Reuel Tolkien <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Hobbit, czyli tam i z powrotem 1937 </li><li> Silmarillion 1977 </li><li> Władca Pierścieni 1954 </li><h2>Autor</h2><li>Aleksander Dumas (ojciec) <img src='http://commons.wikimedia.org/wiki/Special:FilePath/Dumas_by_Nadar,_1855.jpg?width=300'/><br /><b>Lista jego/jej książek</b><li> Hrabia Monte Christo 1844 </li><h2>Autor</h2><li>Tadeusz Dołęga-Mostowicz <img src='http://commons.wikimedia.org